# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

<Client: 'inproc://10.1.0.4/7750/1' processes=1 threads=4, memory=2.00 GB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1040,Dan,-0.5432712477307513,0.3925404442822791
2000-01-01 00:00:01,990,Charlie,0.30289465944930494,-0.7423747981531261
2000-01-01 00:00:02,1030,Bob,-0.9330050818372675,0.4088963757619719
2000-01-01 00:00:03,975,Hannah,0.48738261165213004,-0.09185892629468873
2000-01-01 00:00:04,995,Xavier,0.7238054929314182,0.373999851478499
2000-01-01 00:00:05,959,Quinn,-0.706522095161477,0.05573911967338874
2000-01-01 00:00:06,963,Oliver,-0.2216127681603106,0.45716477928894195
2000-01-01 00:00:07,992,Patricia,-0.5588121125390095,-0.9238633712490585
2000-01-01 00:00:08,984,Dan,0.12399530496513322,-0.6160789638459814


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,960,Ray,0.5385618767596356,0.22762956937419365
2000-01-30 00:00:01,996,Charlie,-0.918558193783136,-0.49443146653558046
2000-01-30 00:00:02,1038,Zelda,-0.5076682243022212,0.08468585042811494
2000-01-30 00:00:03,973,Dan,0.8557651819229879,0.30004992045558665
2000-01-30 00:00:04,1009,Ray,-0.9761966038624998,-0.4721621441093402
2000-01-30 00:00:05,953,Tim,-0.4915646433262675,-0.9071585424826731
2000-01-30 00:00:06,966,Oliver,-0.8930264495242537,0.6722433292993046
2000-01-30 00:00:07,954,George,0.46025840628031833,-0.5830091044467729
2000-01-30 00:00:08,1018,George,-0.7347913942334989,0.6869575650374311


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

timestamp    id     name         x         y
0  2000-01-01 00:00:00  1040      Dan -0.543271  0.392540
1  2000-01-01 00:00:01   990  Charlie  0.302895 -0.742375
2  2000-01-01 00:00:02  1030      Bob -0.933005  0.408896
3  2000-01-01 00:00:03   975   Hannah  0.487383 -0.091859
4  2000-01-01 00:00:04   995   Xavier  0.723805  0.374000

In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
               timestamp     id    name        x        y
npartitions=30                                           
                  object  int64  object  float64  float64
                     ...    ...     ...      ...      ...
...                  ...    ...     ...      ...      ...
                     ...    ...     ...      ...      ...
                     ...    ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [10]:
df.head()

timestamp    id     name         x         y
0  2000-01-01 00:00:00  1040      Dan -0.543271  0.392540
1  2000-01-01 00:00:01   990  Charlie  0.302895 -0.742375
2  2000-01-01 00:00:02  1030      Bob -0.933005  0.408896
3  2000-01-01 00:00:03   975   Hannah  0.487383 -0.091859
4  2000-01-01 00:00:04   995   Xavier  0.723805  0.374000

## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

Dask DataFrame Structure:
                     timestamp     id    name        x        y
npartitions=30                                                 
                datetime64[ns]  int64  object  float64  float64
                           ...    ...     ...      ...      ...
...                        ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.36 s, sys: 334 ms, total: 3.69 s
Wall time: 3.15 s


name
Alice      -0.002143
Bob         0.002917
Charlie     0.000736
Dan        -0.000744
Edith      -0.001497
Frank       0.002565
George      0.001669
Hannah     -0.002771
Ingrid      0.001030
Jerry       0.001051
Kevin       0.001960
Laura      -0.002238
Michael     0.001293
Norbert     0.001943
Oliver     -0.002147
Patricia   -0.001049
Quinn      -0.003255
Ray         0.002198
Sarah       0.002036
Tim        -0.000305
Ursula     -0.001125
Victor      0.004142
Wendy      -0.000333
Xavier      0.002618
Yvonne      0.003469
Zelda      -0.004652
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

Dask DataFrame Structure:
                     timestamp     id    name        x        y
npartitions=30                                                 
                datetime64[ns]  int64  object  float64  float64
                           ...    ...     ...      ...      ...
...                        ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
Dask Name: read-parquet, 30 tasks

In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 942 ms, sys: 133 ms, total: 1.07 s
Wall time: 953 ms


name
Alice      -0.002143
Bob         0.002917
Charlie     0.000736
Dan        -0.000744
Edith      -0.001497
Frank       0.002565
George      0.001669
Hannah     -0.002771
Ingrid      0.001030
Jerry       0.001051
Kevin       0.001960
Laura      -0.002238
Michael     0.001293
Norbert     0.001943
Oliver     -0.002147
Patricia   -0.001049
Quinn      -0.003255
Ray         0.002198
Sarah       0.002036
Tim        -0.000305
Ursula     -0.001125
Victor      0.004142
Wendy      -0.000333
Xavier      0.002618
Yvonne      0.003469
Zelda      -0.004652
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 946 ms, sys: 31.2 ms, total: 977 ms
Wall time: 928 ms


name
Alice      -0.002143
Bob         0.002917
Charlie     0.000736
Dan        -0.000744
Edith      -0.001497
Frank       0.002565
George      0.001669
Hannah     -0.002771
Ingrid      0.001030
Jerry       0.001051
Kevin       0.001960
Laura      -0.002238
Michael     0.001293
Norbert     0.001943
Oliver     -0.002147
Patricia   -0.001049
Quinn      -0.003255
Ray         0.002198
Sarah       0.002036
Tim        -0.000305
Ursula     -0.001125
Victor      0.004142
Wendy      -0.000333
Xavier      0.002618
Yvonne      0.003469
Zelda      -0.004652
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html